## Generacion de resumenes

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

In [91]:
df_full = pd.read_csv('reddit_database_sentiment.csv', sep=";")

C:\Users\iocol\AppData\Local\Temp\ipykernel_13096\2731419115.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('reddit_database_sentiment.csv', sep=";")


In [92]:
df_full

,created_date,created_timestamp,subreddit,title,author,author_created_utc,full_link,score,num_comments,num_crossposts,subreddit_subscribers,post,sentiment
0,2010-02-11 19:47:22,1265910442.0,analytics,So what do you guys all do related to analytic...,xtom,1.227476e+09,https://www.reddit.com/r/analytics/comments/b0...,7.0,4.0,0.0,NaN,There's a lot of reasons to want to know all t...,NEGATIVE
1,2010-03-04 20:17:26,1267726646.0,analytics,"Google's Invasive, non-Anonymized Ad Targeting...",xtom,1.227476e+09,https://www.reddit.com/r/analytics/comments/b9...,2.0,1.0,0.0,NaN,"I'm cross posting this from /r/cyberlaw, hopef...",NEGATIVE
2,2011-01-06 04:51:18,1294282278.0,analytics,"DotCed - Functional Web Analytics - Tagging, R...",dotced,1.294282e+09,https://www.reddit.com/r/analytics/comments/ew...,1.0,1.0,NaN,NaN,"DotCed,a Functional Analytics Consultant, offe...",NEGATIVE
3,2011-01-19 11:45:30,1295430330.0,analytics,Program Details - Data Analytics Course,iqrconsulting,1.288245e+09,https://www.reddit.com/r/analytics/comments/f5...,0.0,0.0,NaN,NaN,Here is the program details of the data analyt...,NEGATIVE
4,2011-01-19 21:52:28,1295466748.0,analytics,potential job in web analytics... need to anal...,therewontberiots,1.278672e+09,https://www.reddit.com/r/analytics/comments/f5...,2.0,4.0,NaN,NaN,i decided grad school (physics) was not for me...,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274234,2022-05-07 21:38:52,1651948732.0,rstats,Help interpretting lmer model output,seeking-stillness,NaN,https://www.reddit.com/r/rstats/comments/ukjiy...,1.0,0.0,0.0,64078.0,Hello! I am wonder how the following output wo...,NEGATIVE
274235,2022-05-07 22:13:52,1651950832.0,rstats,Medical stats book with R,Sweaty_Catch_4275,NaN,https://www.reddit.com/r/rstats/comments/ukk7u...,1.0,0.0,0.0,64080.0,Can anybody recommend me a book with medical s...,POSITIVE
274236,2022-05-08 00:38:50,1651959530.0,rstats,Markov chains with unequal sequence lengths,sebelly,NaN,https://www.reddit.com/r/rstats/comments/ukn1i...,1.0,0.0,0.0,64083.0,I'm trying to build a simple Markov chain. I h...,NEGATIVE
274237,2022-05-08 01:19:00,1651961940.0,rstats,view all available Rcpp::plugins,BOBOLIU,NaN,https://www.reddit.com/r/rstats/comments/uknuh...,1.0,0.0,0.0,64084.0,How do I view all available Rcpp::plugins? Tha...,POSITIVE


In [93]:
texto_prueba = df_full['post'][45]


### Test 1 implementacion basica.

Vamos a hacer una primera implementacion con un texto de prueba para ver si el algoritmo funciona correctamente.

dividimos el proceso en los siguientes pasos:

- preprocesamos el texto eliminando enlaces, numeros convertimos a minuscula, lamatizamos y tokenizamos en oraciones.
- creamos un countvactorizer con las stopwords del texto completo preprocesado, de ese modo podemos ver la frecuencia de las palabras en el texto.
- vamos a darle una puntuacion a cada oración en funcion de la frecuencia de las palabras en cada oración.
- seleccionamos las oraciones con mayor puntuacion y las unimos para formar el resumen.



In [94]:

def get_wordnet_pos(word):
    """Funcion auxiliar para obtener el POS de cada palabra"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_text(text):
    """Funcion para preprocesar el texto devuelve el texto completo preprocesado y las oraciones procesadas"""
    
    # Convertir a minúsculas
    text = text.lower()

    # Eliminar enlaces y números
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\d+', '', text)

    # Tokenizar manteniendo signos de puntuación
    re_sent_tokenizer = RegexpTokenizer(r'\w+|[.!?;:]')
    tokens = re_sent_tokenizer.tokenize(text)

    # Lematizar cada palabra con su POS
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]

    # Unir tokens lematizados en un solo texto
    lemmatized_text = ' '.join(lemmatized_tokens)

    # Tokenizar en frases
    sentences = sent_tokenize(lemmatized_text)

    return sentences, lemmatized_text

In [95]:
raw_sentences = nltk.sent_tokenize(texto_prueba)
procesed_sentences, cleaned_text = preprocess_text(texto_prueba)

In [96]:
# Creamos un count vectorizer con las stopwords del texto completo preprocesado

stop_words = stopwords.words('english')
cv = CountVectorizer(stop_words=stop_words)
cv_matrix = cv.fit_transform([cleaned_text])

In [97]:
# Obtenemos las palabras únicas del texto
unique_words = list(cv.vocabulary_.keys())
# Creamos un diccionario de palabras y sus frecuencias
word_freq = dict(zip(unique_words, cv_matrix.sum(axis=0).A1))

In [98]:
# Calculamos la frecuencia de palabras en cada oración
sentence_scores = {}
for i, sentence in enumerate(procesed_sentences):
    for word, freq in word_freq.items():
        if word in sentence:
            if raw_sentences[i] in sentence_scores.keys():
                sentence_scores[raw_sentences[i]] += freq
            else:
                sentence_scores[raw_sentences[i]] = freq

In [99]:
puntuaciones = np.array(list(sentence_scores.values()))

In [100]:
puntuaciones

array([19, 10,  9, 10, 16, 10,  9,  1])

No podemos seleccionar un numero de oraciones concreto ya que no sabemos cuantas oraciones tiene el texto, Por lo que nos vamos a quedar con las que tengan una puntuacion mayor al percentil 75 de modo que nos quedemos con las oraciones mas relevantes.

In [101]:
threshold = np.quantile(puntuaciones, 0.75)
indices = np.where(puntuaciones > threshold)[0]
slected_raw_sentences = [sent for i, sent in enumerate(raw_sentences) if i in indices]

In [102]:
sumary = "".join(slected_raw_sentences)

In [103]:
sumary

'Until the latest version of Google Analytics, a friend of mine used to receive a report every month with a general overview of various metrics from his website.I basically want Analytics to email a standard report with metrics from the menus Audience, Traffic sources, etc.'

## Test 2 Analisis de los resultados.

El test uno ha funcionado sin embargo vemos un problema a la hora de asignar las puntuaciones a las palabras ya que hay palabras que se repiten mucho pero que no estan aportando demasiado significado por lo que no tienen sentido que tengan una puntuacion tan alta. Asumiendo que los datos del dataset nos estans sesgados de ninguna manera podemos asumir que las palaras mas repetidas son las menos importantes.

In [104]:
# Vamos a hacer un analisis de las palabras mas comunes en el dataset para ver si podemos hacer algo al respecto.

In [105]:
df_full['post'] = df_full['post'].apply(lambda x: str(x))
df_full['post'].sample(100)

#reducto todos los strings de esta serie a un solo string
texto_completo = " ".join(df_full['post'])

In [106]:
re_word_tokenizer = RegexpTokenizer(r'\w+')
words = re_word_tokenizer.tokenize(texto_completo)

In [107]:
text = nltk.Text(words)
fdist = nltk.FreqDist(text)
most_common = fdist.most_common(300)

In [108]:
lista_palaras_comunes = [word.lower() for word, freq in most_common]

In [109]:
# comprobamos que de este modo las palabras que no aportan significado estan en la lista de palabras mas comunes

sentence_scores = {}
for i, sentence in enumerate(procesed_sentences):
    for word, freq in word_freq.items():
        if word in sentence:
            if word in lista_palaras_comunes:
                print(word)
            elif raw_sentences[i] in sentence_scores.keys():
                sentence_scores[raw_sentences[i]] += freq
            else:
                sentence_scores[raw_sentences[i]] = freq
puntuaciones = np.array(list(sentence_scores.values()))
puntuaciones

use
go
look
example
found
go
new
help
set
go
want
etc
much
like
one
help
one
someone
point
right
thank


array([17,  5,  8,  6, 14,  7,  2])

Aunque el resultado no es perfecto ya que el dataset parece estar sesgado hacie el ambito cientifico si que sonseguimos quitar muchas palabras que no estana aportando mucho significado lo cual nos será util para la implementacion final.

## Implementacion final

vamos a hacer una implementacion final en forma de funcion.

In [110]:
def post_summarisation(text: str):
    """Function to summarize a text"""

    raw_sentences = nltk.sent_tokenize(text)
    procesed_sentences, cleaned_text = preprocess_text(text)

    # Utilizamos un Countvectorizer con las stopwords del texto completo preprocesado
    stop_words = stopwords.words('english')
    cv = CountVectorizer(stop_words=stop_words)
    cv_matrix = cv.fit_transform([cleaned_text])

    # Obtner las palabras únicas del texto
    unique_words = list(cv.vocabulary_.keys())
    # Creamos un diccionario de palabras y sus frecuencias
    word_freq = dict(zip(unique_words, cv_matrix.sum(axis=0).A1))

    # Calculamos la frecuencia de palabras en cada oración
    sentence_scores = {}
    for i, sentence in enumerate(procesed_sentences):
        for word, freq in word_freq.items():
            if word in sentence and word not in lista_palaras_comunes:
                raw_sentence = raw_sentences[i] if i < len(raw_sentences) else ""
                if raw_sentence in sentence_scores.keys():
                    sentence_scores[raw_sentence] += freq
                else:
                    sentence_scores[raw_sentence] = freq

    puntuaciones = np.array(list(sentence_scores.values()))
    threshold = np.quantile(puntuaciones, 0.75)
    indices = np.where(puntuaciones > threshold)[0]
    slected_raw_sentences = [sent for i, sent in enumerate(raw_sentences) if i in indices]

    sumary = "".join(slected_raw_sentences)

    return sumary

# Ejemplo
texto_prueba = df_full['post'][45]
resumen = post_summarisation(texto_prueba)
print(resumen)

Until the latest version of Google Analytics, a friend of mine used to receive a report every month with a general overview of various metrics from his website.I basically want Analytics to email a standard report with metrics from the menus Audience, Traffic sources, etc.


In [111]:
texto_prueba = df_full['post'][45]

In [112]:
resumen = post_summarisation(texto_prueba)

In [113]:
resumen

'Until the latest version of Google Analytics, a friend of mine used to receive a report every month with a general overview of various metrics from his website.I basically want Analytics to email a standard report with metrics from the menus Audience, Traffic sources, etc.'

In [114]:
samples = df_full['post'].sample(10).to_list()

In [115]:
for sample in samples:
    resumen = post_summarisation(sample)
    print("\nTexto original:\n")
    print(sample)
    print("\nResumen:\n")
    print(resumen)
    print("\n" * 5)


Texto original:

The role that data analytics plays in modern business is becoming increasingly appreciated. According to one report, the per-dollar-spent ROI gained from using analytics &amp; increased from[ $10.66 in 2011 to $13.01 in 2014](https://nucleusresearch.com/research/single/analytics-pays-back-13-01-for-every-dollar-spent/). Working with analytics is one thing, but translating data-driven insights into useful work products is quite another. That’s where[ data visualization](https://www.inzata.com/bi-encyclopedia/data-visualization/) enters the picture.[ Data visualization](https://www.inzata.com/bi-encyclopedia/data-visualization/) is an opportunity to go beyond dumping data into an Excel spreadsheet. With the right approach, data[ visualizations](https://www.inzata.com/bi-encyclopedia/visualizations/) can improve a company’s efficiency and effectiveness in the following ways.

### Shorter and Better Meetings

At many organizations, analytics need to be converted into work